In [ ]:
#importing necessary package
import pandas as pd

In [ ]:
#loading given data set
data_frame1 = pd.read_csv("data-breastCancer.csv", index_col=0)

In [ ]:
data_frame1.head()

In [ ]:
data_frame1.tail()

In [ ]:
data_frame1.info()

In [ ]:
data_frame1.shape

In [ ]:
data_frame1.describe()

In [ ]:
data_frame1.columns

In [ ]:
from numpy.ma.core import count
count(data_frame1.columns)

In [ ]:
data_frame2= data_frame1.drop(['Unnamed: 32'],axis=1)

In [ ]:
data_frame2

In [ ]:
data_frame2.shape

In [ ]:
data_frame2.columns

In [ ]:
data_frame2.dtypes

In [ ]:
data_frame2['diagnosis_value'] = data_frame2['diagnosis'].map({'M':1, 'B':0})

In [ ]:
data_frame2.columns

In [ ]:
data_frame2 = data_frame2.drop(['diagnosis'],axis=1)

In [ ]:
data_frame2.dtypes

In [ ]:
data_frame2.shape

In [ ]:
data_frame2.isnull().sum()

In [ ]:
#importing seaborn package
import seaborn as sns
sns.countplot(x='diagnosis_value', data=data_frame2)
print(data_frame2['diagnosis_value'].value_counts())
X = data_frame2.drop(labels='diagnosis_value', axis=1)
y = data_frame2['diagnosis_value']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import numpy as np
RandomFirst_C = RandomForestClassifier(random_state=42)
np.random.seed(42)
Cross_Val = cross_validate (RandomFirst_C, X_train, y_train, cv = 10)

In [ ]:
print("10-fold cross val accuracy score: ", Cross_Val['test_score'])
print("cross val accuracy mean score: ", Cross_Val['test_score'].mean())

In [ ]:
param_d = {'max_depth': [2, 3, 4],
              'bootstrap': [True, False],
              'max_features': ['auto', 'sqrt', 'log2', None],
              'criterion': ['gini', 'entropy']}

In [ ]:
from sklearn.model_selection import GridSearchCV
RandomForest_CV = GridSearchCV(RandomFirst_C , cv = 10,
                     param_grid=param_d, 
                     n_jobs = 3)
RandomForest_CV.fit(X_train, y_train)

In [ ]:
print('Best Parameters using grid search: {0}'.format(RandomForest_CV.best_params_))

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
baggingClass_fier = BaggingClassifier()
ClassV = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
score_s = cross_val_score(baggingClass_fier, X, y, scoring='roc_auc', cv=ClassV, n_jobs=-1)

In [ ]:
from numpy import mean
print('BaggingClassifier model Mean: %.2f' % mean(score_s))

In [ ]:
cross_valid = cross_validate (baggingClass_fier, X_train, y_train, cv = 10)
print("Tenfold cross validation accuracy score: ", cross_valid['test_score'])
print("Accuracy mean score bagging classifier score: ", cross_valid['test_score'].mean())

In [ ]:
baggingClass_fier.fit(X_train, y_train)
#Prediction on test set by using model
pred = baggingClass_fier.predict(X_test)
#probability of classes in the test set observation
predictionProbability = baggingClass_fier.predict_proba (X_test)
print ('class_0','\t', 'class_1')
print(predictionProbability[:5])

In [ ]:
#Accuracy scores of test set
print('BaggingClassifier model Accuracy of the selected model in the test set: {:.2f}'.format(baggingClass_fier.score(X_test, y_test)))

In [ ]:
import matplotlib.pyplot as plyt
#confusion matrix
from sklearn.metrics import confusion_matrix
confusionMatrix = confusion_matrix(y_test, pred )
sns.heatmap(confusionMatrix, annot = True)
plyt.xlabel ('Values predicted')
plyt.ylabel ('actual values')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
pred_prob_c1 = predictionProbability[:,1]
fpr, tpr, threshold = roc_curve (y_test,pred_prob_c1, pos_label = 1)
roc_auc_score = roc_auc_score (y_test,pred_prob_c1)
print("BaggingClassifier model roc auc score {:.2f}".format(roc_auc_score))

In [ ]:
# Visualization of ROC score
plyt.subplots(figsize = (20,5))
plyt.plot( fpr, tpr, label='BaggingClassifier model Test AUC: %0.2f'%roc_auc_score)
plyt.plot([0,1], '--')
plyt.xlabel('BaggingClassifier model False Positive Rate')
plyt.ylabel('BaggingClassifier model True Positive Rate')
plyt.title ("BaggingClassifier model ROC Curve (AUC = {:.2f})".format(roc_auc_score))
plyt.grid()
plyt.show();

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_grid = {'n_estimators' :[int(x) for x in np.linspace(start = 100, stop = 1000, num = 10, endpoint = True)],
             'max_features': ['auto', 'sqrt', 'log2'],
             'max_depth': [int(x) for x in np.linspace(start = 3, stop = 36, num=33, endpoint = True)],
             'min_samples_split': [5, 10, 15],
             'min_samples_leaf': [3, 4, 5], 
             'bootstrap': ['True']}
print(random_grid)

In [ ]:
#tuning for base randomforest model
tune_base_rf = RandomForestClassifier(random_state=42)
randomSearch_rf = RandomizedSearchCV (estimator=tune_base_rf, param_distributions  = random_grid, random_state=42, cv = 10, n_iter=100)
randomSearch_rf.fit(X_train, y_train)
randomSearch_rf.best_params_

In [ ]:
tuned_rf = RandomForestClassifier (n_estimators = 100, min_samples_split = 5, min_samples_leaf = 3, max_features = 'auto',  max_depth = 26, bootstrap = True, random_state=42)
tuned_cv = cross_validate (tuned_rf, X_train, y_train, cv = 10)
print("Accuracy score: ", tuned_cv['test_score'])
print("Accuracy mean score of CV: ", tuned_cv['test_score'].mean())

In [ ]:
# Fitting the model into test set
tuned_rf.fit(X_train, y_train)

In [ ]:
#Prediction on test set using model
pred = tuned_rf.predict(X_test)
#probability of classes in the test set observation
pred_prob = tuned_rf.predict_proba (X_test)
print ('class_0','\t', 'class_1')
print(pred_prob[:5])

In [ ]:
#accuracy scores of test set
print('RandomForestClassifier model Accuracy of the selected model in the test set: {:.4f}'.format(tuned_rf.score(X_test, y_test)))

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, pred )
sns.heatmap(conf_matrix, annot = True)
plyt.xlabel ('predicted values')
plyt.ylabel ('actual values')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
pred_prob_c1 = pred_prob[:,1]
fpr, tpr, threshold = roc_curve (y_test,pred_prob_c1, pos_label = 1)
roc_auc_score = roc_auc_score (y_test,pred_prob_c1)
print("Roc AUC Score : ",round(roc_auc_score,2))

In [ ]:
plyt.subplots(figsize = (20,5))
plyt.plot( fpr, tpr, label='Test AUC: %0.2f'%roc_auc_score)
plyt.plot([0,1], '--')
plyt.xlabel('False Positive Rate')
plyt.ylabel('RandomForestClassifier model True Positive Rate')
plyt.title ("RandomForestClassifier model ROC Curve (AUC = {:.2f})".format(roc_auc_score))
plyt.grid()
plyt.show();